In [1]:
import qlib
from pathlib import Path
from qlib.constant import REG_CN  # 'cn'
from qlib.data import D
from qlib.data.filter import ExpressionDFilter,NameDFilter

# 在这里定义路径
# 当前绝对路径
path_cur = Path.cwd() # D:/github/qlib/playGround/jupyter
# 项目根目录
path_project_root = path_cur.parent.parent # 绝对路径 ./qlib

# 数据路径
path_data = path_project_root / Path('./.qlib/qlib_bin')


ModuleNotFoundError: No module named 'qlib'

In [ ]:
# In order to get the data, users need to initialize Qlib with qlib.init first
qlib.init(default_conf="client", provider_uri=path_data, region=REG_CN)


[85140:MainThread](2023-03-02 17:04:31,156) INFO - qlib.Initialization - [config.py:415] - default_conf: client.
[85140:MainThread](2023-03-02 17:04:31,161) INFO - qlib.Initialization - [__init__.py:74] - qlib successfully initialized based on client settings.
[85140:MainThread](2023-03-02 17:04:31,162) INFO - qlib.Initialization - [__init__.py:76] - data_path={'__DEFAULT_FREQ': WindowsPath('D:/github/qlib/.qlib/qlib_bin')}


In [ ]:
# 从calendar中读取 指定的频率和时间范围，查看数据的时间范围
start,end = '2022-01-01','2023-02-28'
cal_d = D.calendar(start_time=start, end_time=end, freq='day')
print(cal_d[:2])

# 初始化一个stook pool 从instruments中，指定 market 为文件名，读取个股code
mkt_csi300 = D.instruments(market='csi300')   # {'market': 'all', 'filter_pipe': []}
mkt_csi300_list = D.list_instruments(instruments=mkt_csi300, start_time=start, end_time=end, as_list=True)

# 有filter api 用来过滤数据
# expressionDFilter = ExpressionDFilter(rule_expression='$close>2000')
# instruments = D.instruments(market='csi300', filter_pipe=[expressionDFilter])
# D.list_instruments(instruments=instruments, start_time='2020-01-01', end_time='2020-09-25', as_list=True)


[Timestamp('2023-01-03 00:00:00') Timestamp('2023-01-04 00:00:00')]


In [ ]:
# # 加载数据
# fields = ['$open','$close','$high','$low', '$volume', 'Ref($close, 1)', 'Mean($close, 3)', '$high-$low']
# data = D.features(mkt_csi300, fields, start_time=start, end_time=end, freq='day')
# data

```python
# 对象初始化
class_x_config = {
        "class": "Class_X", # 类命
        "module_path": "qlib.xx.xx.xx", # 该类所在的路径
        # 该对象的超参数
        "kwargs": {
            'param_1': 'value_1',
            'param_2': 'value_2',
            ...
            'param_n': 'value_n',
        },
}
```

In [ ]:
# 训练参数设置
# Qlib要求我们使用Dict类型变量，以一种约定俗成的定义方式来进行参数初始化

# 数据处理类handler设置
data_handler_config = {
    "start_time": start,
    "end_time": end,
    "fit_start_time": start,
    "fit_end_time": "2022-12-31",
    "instruments": "csi300" # 使用这段时间内中证三百指数成分股进行交易
}

# 训练任务参数设置
task = {
    # 使用的model的参数设置
    "model": {
        "class": "XGBModel", # 原代码中使用LGBM，这里改成XGB
        "module_path": "qlib.contrib.model.xgboost",
        # 模型的超参数
        "kwargs": {
            "loss": "mse",
            "colsample_bytree": 0.8879,
            "learning_rate": 0.0421,
            "max_depth": 8,
            'tree_method': 'gpu_hist',
            'gpu_id': 0
        },
    },

    # 为model提供数据的dataset类的参数设置
    "dataset": {
        "class": "DatasetH", 
        "module_path": "qlib.data.dataset",
        "kwargs": {
            "handler": {
                "class": "Alpha158",
                "module_path": "qlib.contrib.data.handler",
                "kwargs": data_handler_config,   # 使用上边定义的data_handler_config
            },
            "segments": {
                "train": ("2008-01-01", "2014-12-31"), # 训练集时间范围
                "valid": ("2015-01-01", "2016-12-31"), # 验证集时间范围
                "test": ("2017-01-01", "2020-08-01"),  # 测试集时间范围
            },
        },
    },
}


In [ ]:
# 初始化模型和数据集
from qlib.utils import init_instance_by_config

model = init_instance_by_config(task["model"])
dataset = init_instance_by_config(task["dataset"])


In [ ]:
# 模型训练
# start exp to train model
from qlib.workflow import R
from qlib.utils import flatten_dict


with R.start(experiment_name="train_model"):
    R.log_params(**flatten_dict(task))
    model.fit(dataset)  # 模型训练
    R.save_objects(trained_model=model)
    rid = R.get_recorder().id # 获得这个R中的recorder的id，便于之后取用这个recorder